In [1]:
__author__ = "me"
__date__ = "2015_10_13"

%pylab inline
import pandas as pd
import numpy as np
import geopandas as gp

import pylab as plt
import os

import requests
s = requests.get("https://raw.githubusercontent.com/Casyfill/CUSP_templates/master/Py/fbMatplotlibrc.json").json()
plt.rcParams.update(s)


Populating the interactive namespace from numpy and matplotlib


/Users/casy/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:855: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## PropID

In [2]:
PARQA = os.getenv('PARQA')

In [21]:
prop = pd.read_excel(PARQA + 'data/Input/Parks_Data/CUSP_Adjusted_Spatial_Data.xlsx')[['ParkID','Name']]

In [24]:
prop = prop.dropna()

In [30]:
def trySplit(x, spl='-'):
    if spl in x:
        return x.split(spl)[0].strip()
    else:
        return x

In [31]:
prop.ParkID = prop.ParkID.apply(trySplit)
prop.Name = prop.Name.apply(lambda x: trySplit(x, spl='Zone'))

In [45]:
prop = prop.drop_duplicates().rename(columns={'Name':'NAME'})
prop.NAME = prop.NAME.str.lower()

In [56]:
prop[prop.NAME.str.contains('prospect')]

,ParkID,NAME
28,B159,mt prospect park
181,X010,prospect park (2)
347,B073,prospect park west
350,B073,prospect park southwest upper
351,B073,prospect park southwest lower
370,B073,prospect lake near duck island
371,B073,prospect lake near west island
1277,B159,mount prospect park
2066,X148H1,prospect playground


In [ ]:
## AGGREGATION

## 311 Property Names

In [40]:
onto = pd.read_csv(PARQA + 'parqa/311/ONTOLOGY/onto_data/Ontology_matched.csv', index_col=0)

In [47]:
onto1 = onto.merge(prop, on='NAME', how='left')

In [51]:
print len(onto)

1793


In [50]:
print len(onto1[pd.isnull(onto1.ParkID)])

616


In [57]:
onto1[pd.isnull(onto1.ParkID)].head()

,cleanName,NAME,Type,valid,parkDistrict,lat,lon,ParkID
9,prospect park - east parade grounds,prospect park,empiric,av,B-19,-73.969682,40.661353,NaN
10,central park - east 96th street playground,central park,empiric,av,M-13,-73.965558,40.782484,NaN
14,col david marcus playground,col david marcus mem playground,pgs,av,B-12,-73.969600,40.609700,NaN
18,mccarren park - vincent v abate playground,vincent v abate playground,pgs,av,B-01,-73.949800,40.721300,NaN
21,forest park - giovannelli playground,forest park,empiric,av,Q-16,-73.853003,40.702053,NaN


In [61]:
onto_r = onto1[pd.notnull(onto1.ParkID)]

## Matching calls

In [60]:
calls = pd.read_csv(PARQA + 'data/311/311DPR.csv', encoding='utf8', na_values='Unspecified')

myCalls = calls[['Park Facility Name','Descriptor','Created Date','Closed Date','Longitude','Latitude','Location Type', 'Complaint Type']]
myCalls['Park Facility Name'] = myCalls['Park Facility Name'].str.lower()
myCalls['Park Facility Name'].head()

namedCalls = myCalls[pd.notnull(myCalls['Park Facility Name'])]

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [62]:
ncMatched = namedCalls.merge(onto_r, how='left',left_on='Park Facility Name', right_on='cleanName')
ncMatched.head(1)

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,Complaint Type,cleanName,NAME,Type,valid,parkDistrict,lat,lon,ParkID
0,geo soilan park - battery park city,Graffiti or Vandalism,12/31/2010 04:31:52 PM,12/31/2010 05:36:58 PM,NaN,NaN,Park,Maintenance or Facility,geo soilan park - battery park city,battery park city,other,av,M-01,-74.016893,40.711882,M283


In [63]:
ncMatched[['Created Date', 'ParkID','Descriptor','Complaint Type']].to_csv(PARQA + '/data/311/calls_matched_property.csv')